<a href="https://colab.research.google.com/github/asmipulgam/RAG/blob/main/RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U \
sentence-transformers==3.0.1 \
langchain==0.3.19 \
langchain-community==0.3.18 \
langchain-groq==0.2.4 \
langchain-huggingface==0.1.2 \
einops==0.8.1 \
faiss-cpu==1.10.0

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [3]:
import getpass
import os

In [4]:
os.environ["GROQ_API_KEY"]=getpass.getpass()

··········


In [5]:
os.environ["HP_TOKEN"]=getpass.getpass()

··········


In [6]:
def pretty_print_docs(docs):
  for i , d  in enumerate(docs):
    print(f"{'-'*50}\nDocument{i+1}:")
    print(f"Content:\n{d.page_content}\n")
    print("Metadata:")
    for key, value in d.metadata.items():
      print(f" {key}:{value}")
  print(f"{'-'*50}")



In [7]:
!pip install pypdf

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import faiss
from langchain_text_splitters import RecursiveCharacterTextSplitter
file_path="/content/TPC_H_Data_into_PgAdmin+Queries+Solution.pdf"
loader=PyPDFLoader(file_path)
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts=text_splitter.split_documents(documents)
for idx, text in enumerate(texts):
  text.metadata["id"]=idx

In [9]:
print(documents)

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-09-03T12:49:24-07:00', 'author': 'Bharatesh Chakravarthi', 'moddate': '2025-09-03T12:49:24-07:00', 'source': '/content/TPC_H_Data_into_PgAdmin+Queries+Solution.pdf', 'total_pages': 28, 'page': 0, 'page_label': '1'}, page_content='Term: Fall 2025     Subject: Computer Science & Engineering (CSE)   Number: 512 \nCourse Title: Distributed Database Systems (CSE 512) \nTPC-H Data into PgAdmin \n \nStep 1: Start PostgreSQL and pgAdmin \n1. Ensure PostgreSQL is installed on your system. \n2. Open pgAdmin and connect to your PostgreSQL instance. \n \nStep 2: Create a New Database \n1. In pgAdmin, navigate to Servers → PostgreSQL → Databases. \n2. Right-click on Databases → Create → Database.... \n3. Name your database (e.g., tpch). \n4. Click Save. \n \nStep 3: Create Tables \n1. Open Query Tool in pgAdmin. \n2. Open and run the create_tables.sql file: \no 

In [10]:
%pip install -qU langchain-community pypdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires langchain-core<1.0.0,>=0.3.35, but you have langchain-core 1.2.0 which is incompatible.
langchain 0.3.19 requires langchain-text-splitters<1.0.0,>=0.3.6, but you have langchain-text-splitters 1.1.0 which is incompatible.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 1.2.0 which is incompatible.
langchain-groq 0.2.4 requires langchain-core<0.4.0,>=0.3.33, but you have langchain-core 1.2.0 which is incompatible.


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
embedding=HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs={'trust_remote_code':True})
retriever=FAISS.from_documents(texts,embedding).as_retriever(search_kwargs={"k":20})

In [15]:
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
query="what in this text file?"
llm=ChatGroq(
    model="openai.gpt-oss-120b",
    temperature=0.5
)
chain=RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
response=chain.invoke(query)
print(response)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `openai.gpt-oss-120b` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'code': 'model_not_found'}}